In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import sys
sys.path.append('../')
import src.cleaning_functions as cf
import pyjsonviewer
from functools import reduce
import operator

In [2]:
load_dotenv()

True

#### Utilizo los token de la API de Foursquare para poder llamarla 

In [3]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
***30% of the company staff have at least 1 child.**
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
***Account managers need to travel a lot.**
***Everyone in the company is between 25 and 40, give them some place to go party.**
***The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

## MADRID

#### Utilizo una función para buscar las coordenadas del punto concreto elegido para la ciudad de Madrid

In [5]:
todo= {"nombre": ["name"], "dirección": ["location", "address"], "latitud": ["location", "lat"], "longitud": ["location", "lng"]}

In [6]:
cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

In [7]:
Madrid = cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

In [8]:
Madrid

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

### Airports

In [9]:
parametros_airp = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [10]:
airports = requests.get(url_query, params = parametros_airp).json()

In [11]:
airports

{'meta': {'code': 200, 'requestId': '618fc6d1719bbf73443acbec'},
 'response': {'venues': [{'id': '4afde737f964a520e72b22e3',
    'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
    'contact': {},
    'location': {'address': 'Av. de la Hispanidad, s/n',
     'lat': 40.4902221628413,
     'lng': -3.569434148638026,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.4902221628413,
       'lng': -3.569434148638026}],
     'distance': 11590,
     'postalCode': '28042',
     'cc': 'ES',
     'city': 'Madrid',
     'state': 'Madrid',
     'country': 'España',
     'formattedAddress': ['Av. de la Hispanidad, s/n',
      '28042 Madrid Madrid',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d1ed931735',
      'name': 'Airport',
      'pluralName': 'Airports',
      'shortName': 'Airport',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
       'suffix': '.png'},
      'primary': True}],
  

In [12]:
airports.keys()

dict_keys(['meta', 'response'])

In [13]:
airports["response"].keys()

dict_keys(['venues'])

In [14]:
airports["response"]["venues"][0]["name"]

'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)'

In [15]:
data = airports["response"]["venues"]

In [16]:
df = pd.DataFrame(data)
df

,id,name,contact,location,categories,verified,stats,beenHere,hereNow,referralId,venueChains,hasPerk,venuePage
0,4afde737f964a520e72b22e3,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,{},"{'address': 'Av. de la Hispanidad, s/n', 'lat'...","[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 12, 'summary': '12 people are here',...",v-1636812497,[],False,NaN
1,4f590f2ce4b0cd6f8d6828e8,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,{},"{'address': 'C. Gran Via de Hortaleza 49', 'la...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
2,4bc2d53c2a89ef3bc242f488,Metro Aeropuerto T1-T2-T3,{},"{'address': 'Terminal 2', 'crossStreet': 'MAD ...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
3,5b982c4cba57b4002c48bfe5,Transfer Aeropuerto Madrid - Transporte Coyoacan,{},"{'address': 'Calle LIBERTAD, 4', 'lat': 40.420...","[{'id': '54541b70498ea6ccd0204bff', 'name': 'T...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
4,4e24dcb788772beda9dcbc40,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,{},"{'address': 'Paseo del Prado', 'lat': 40.41904...","[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
5,5f574011a4af211b8cd1d400,Aeropuerto Madrid Barajas,{},{'address': 'Calle del Príncipe de Vergara 53'...,"[{'id': '53fca564498e1a175f32528b', 'name': 'T...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,{'id': '591746729'}
6,4bc2d4ab2a89ef3b6742f488,Cercanías Aeropuerto T4,{},"{'address': 'Terminal 4', 'crossStreet': 'Aero...","[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
7,4d1db7ed756e8cfaadcd5154,Parada O'Donnell - Línea Exprés Aeropuerto,{},"{'address': 'Calle de O'Donnell 61', 'lat': 40...","[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
8,5085849be88982ac6a09d6a4,Metro Aeropuerto T4,{},"{'address': 'Terminal 4', 'crossStreet': 'Aero...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN
9,4bc70ab6af07a5930b647e2d,Aeropuerto de Madrid-Cuatro Vientos (MCV),{},{'address': 'Ctra de Acceso al Barrio de la Fo...,"[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636812497,[],False,NaN


In [17]:
type(airports)

dict

In [18]:
airports["response"]["venues"][0]

{'id': '4afde737f964a520e72b22e3',
 'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
 'contact': {},
 'location': {'address': 'Av. de la Hispanidad, s/n',
  'lat': 40.4902221628413,
  'lng': -3.569434148638026,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.4902221628413,
    'lng': -3.569434148638026}],
  'distance': 11590,
  'postalCode': '28042',
  'cc': 'ES',
  'city': 'Madrid',
  'state': 'Madrid',
  'country': 'España',
  'formattedAddress': ['Av. de la Hispanidad, s/n',
   '28042 Madrid Madrid',
   'España']},
 'categories': [{'id': '4bf58dd8d48988d1ed931735',
   'name': 'Airport',
   'pluralName': 'Airports',
   'shortName': 'Airport',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
 

In [19]:
aeropuerto = airports["response"]["venues"]

In [20]:
aeropuerto

[{'id': '4afde737f964a520e72b22e3',
  'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
  'contact': {},
  'location': {'address': 'Av. de la Hispanidad, s/n',
   'lat': 40.4902221628413,
   'lng': -3.569434148638026,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.4902221628413,
     'lng': -3.569434148638026}],
   'distance': 11590,
   'postalCode': '28042',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Av. de la Hispanidad, s/n',
    '28042 Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1ed931735',
    'name': 'Airport',
    'pluralName': 'Airports',
    'shortName': 'Airport',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,

In [21]:
extraer_air = cf.extraetodo(aeropuerto)

In [22]:
extraer_air

[{'nombre': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
  'latitud': 40.4902221628413,
  'longitud': -3.569434148638026,
  'location': {'type': 'Point',
   'coordinates': [40.4902221628413, -3.569434148638026]}},
 {'nombre': 'Metro Madrid Línea 8 (Nuevos Ministerios - Aeropuerto)',
  'latitud': 40.445477752377386,
  'longitud': -3.691766929219179,
  'location': {'type': 'Point',
   'coordinates': [40.445477752377386, -3.691766929219179]}},
 {'nombre': 'Metro Aeropuerto T1-T2-T3',
  'latitud': 40.4682787651721,
  'longitud': -3.5712555983647887,
  'location': {'type': 'Point',
   'coordinates': [40.4682787651721, -3.5712555983647887]}},
 {'nombre': 'Transfer Aeropuerto Madrid - Transporte Coyoacan',
  'latitud': 40.420315457,
  'longitud': -3.69735505,
  'location': {'type': 'Point', 'coordinates': [40.420315457, -3.69735505]}},
 {'nombre': 'Parada Plaza De Cibeles - Línea Exprés Aeropuerto',
  'latitud': 40.41904743901457,
  'longitud': -3

In [23]:
df_airp = pd.DataFrame(extraer_air)
df_airp

,nombre,latitud,longitud,location
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
1,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,40.445478,-3.691767,"{'type': 'Point', 'coordinates': [40.445477752..."
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765..."
3,Transfer Aeropuerto Madrid - Transporte Coyoacan,40.420315,-3.697355,"{'type': 'Point', 'coordinates': [40.420315457..."
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439..."
5,Aeropuerto Madrid Barajas,40.432060,-3.679915,"{'type': 'Point', 'coordinates': [40.43205988,..."
6,Cercanías Aeropuerto T4,40.491486,-3.593721,"{'type': 'Point', 'coordinates': [40.491486343..."
7,Parada O'Donnell - Línea Exprés Aeropuerto,40.421182,-3.669248,"{'type': 'Point', 'coordinates': [40.421181612..."
8,Metro Aeropuerto T4,40.491506,-3.593771,"{'type': 'Point', 'coordinates': [40.491506396..."
9,Aeropuerto de Madrid-Cuatro Vientos (MCV),40.369272,-3.778696,"{'type': 'Point', 'coordinates': [40.369272082..."


In [24]:
len(df_airp)

30

In [25]:
aeropuerto_madrid = df_airp.copy()

### Party

In [26]:
parametros_party = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Discoteca"
}

In [27]:
disco = requests.get(url_query, params = parametros_party).json()

In [28]:
disco["response"].keys()

dict_keys(['venues'])

In [29]:
discoteca = disco["response"]["venues"]

In [30]:
extraer_disco = cf.extraetodo(discoteca)

In [31]:
df_disco = pd.DataFrame(extraer_disco)
df_disco.head()

,nombre,latitud,longitud,location
0,Discoteca Nells Madrid,40.439664,-3.682127,"{'type': 'Point', 'coordinates': [40.439664, -..."
1,Discoteca Palace,40.418451,-3.709102,"{'type': 'Point', 'coordinates': [40.418450979..."
2,Discoteca Almontes Sala Rociera,40.432799,-3.678725,"{'type': 'Point', 'coordinates': [40.43279897,..."
3,la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -..."
4,Discoteca BAO-BAB Madrid,40.453919,-3.677663,"{'type': 'Point', 'coordinates': [40.453919368..."


### Vegan Restaurants

In [32]:
parametros_vegan = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "vegan"
}

In [33]:
vegan = requests.get(url_query, params = parametros_vegan).json()

In [34]:
vegano = vegan["response"]["venues"]

In [35]:
extraer_vegan = cf.extraetodo(vegano)

In [36]:
df_vegan = pd.DataFrame(extraer_vegan)
df_vegan.head()

,nombre,latitud,longitud,location
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ..."
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ..."
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -..."
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -..."
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -..."


### Nursery

In [37]:
parametros_nurs = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "guarderías"
}

In [38]:
nursery = requests.get(url_query, params = parametros_nurs).json()

In [39]:
guarderia = nursery["response"]["venues"]
guarderia

[{'id': '4dfb84981f6eeef806ab25a2',
  'name': 'Guarderia San Nicolas',
  'contact': {},
  'location': {'lat': 40.421913,
   'lng': -3.668392,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.421913,
     'lng': -3.668392}],
   'distance': 2980,
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Madrid Madrid', 'España']},
  'categories': [],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount': 0},
  'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []},
  'referralId': 'v-1636812499',
  'venueChains': [],
  'hasPerk': False},
 {'id': '50a50321e4b0acf883d480ab',
  'name': 'Guardería San Roque',
  'contact': {},
  'location': {'address': 'Calle De La Oca 35',
   'lat': 40.38952195045214,
   'lng': -3.736509681444763,
   'labeledLatLngs': [{'label': 'display

In [40]:
extraer_nursery = cf.extraetodo(guarderia)

In [41]:
df_nursery = pd.DataFrame(extraer_nursery)
df_nursery.head()

,nombre,latitud,longitud,location
0,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -..."
1,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950..."
2,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239..."
3,Guarderia santa isabel,40.542532,-3.637349,"{'type': 'Point', 'coordinates': [40.542531541..."


### unión de las 4 listas de requerimientos para Madrid

In [42]:
MADRID = cf.suma_requisitos(extraer_air,extraer_disco,extraer_vegan,extraer_nursery)

In [46]:
df_MADRID = pd.DataFrame(MADRID)
df_MADRID

,nombre,latitud,longitud,location
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
1,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,40.445478,-3.691767,"{'type': 'Point', 'coordinates': [40.445477752..."
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765..."
3,Transfer Aeropuerto Madrid - Transporte Coyoacan,40.420315,-3.697355,"{'type': 'Point', 'coordinates': [40.420315457..."
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439..."
...,...,...,...,...
89,Encuentro Vegano,40.406691,-3.639279,"{'type': 'Point', 'coordinates': [40.406691, -..."
90,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -..."
91,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950..."
92,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239..."


## MALLORCA

In [50]:
Palma = cf.geocode("P.º de Mallorca, 7, 07011 Palma, Islas Baleares")

In [51]:
Palma

{'type': 'Point', 'coordinates': ['39.57355', '2.64177']}

In [53]:
parametros_airp_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [54]:
airports = requests.get(url_query, params = parametros_airp_p).json()

### Exporto en MongoDB

In [ ]:
#with open('aeropuerto_madrid.json', 'w') as f: #creo un json vacío donde voy a escribir (w) y donde voy a cargar mi lista de diccionarios "extraer_air"
    #json.dump(airports, f)

In [ ]:
pyjsonviewer.view_data(json_data=resp)